# `Parsing Nested JSON Stock Data example:`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

<font size=5>**`Help Support The Channel: Patreon:`**</font><font size=4>[@mrfugudatasci](https://www.patreon.com/mrfugudatasci)</font>

<font size=4>**`Or Consider: Buy Me A Coffee`**[@mrfugudatasci](https://www.patreon.com/mrfugudatasci)</font>

In [191]:
import pandas as pd
import json

# ` These are the data:`

+ What do we notice?
    + Each stock `ticker`, has a list of values as dictionaries and contains more dictionaries
    + Inside the list: we have 2 dictionaries which look similar
        + These are distinguished by the time `t`
            + We have: `o` for open price
            + `h` for high price
            + `l` low price
            + `c`, closing price
            + `v` volume

In [189]:

stocks={
    "AAPL": [
        {
            "t": 1610570640,
            "o": 131.11,
            "h": 131.12,
            "l": 131.02,
            "c": 131.03,
            "v": 11892
        },
        {
            "t": 1610570700,
            "o": 131.05,
            "h": 131.07,
            "l": 130.98,
            "c": 131.05,
            "v": 8640
        }
    ],"ADBE": [
        {
            "t": 1610570640,
            "o": 472.96,
            "h": 472.96,
            "l": 472.8,
            "c": 472.82,
            "v": 819
        },
        {
            "t": 1610570700,
            "o": 472.8,
            "h": 472.97,
            "l": 472.8,
            "c": 472.97,
            "v": 910
        }
    ],"ADI": [
        {
            "t": 1610570640,
            "o": 158.68,
            "h": 158.715,
            "l": 158.61,
            "c": 158.61,
            "v": 985
        },
        {
            "t": 1610570700,
            "o": 158.57,
            "h": 158.595,
            "l": 158.57,
            "c": 158.595,
            "v": 611
        }
    ] }


# `What is the objective?`
+ we need to pare these, but let's do it 3 different ways. Because after all you never know when one arrangement of your data are better than the other. Or preferred, by a client.

# `Example 01: Parse So stockas are 1 column and rest of data as separate columns`

In [250]:
stock_dta=[]

for i in stocks.items(): # convert to tuples for ease
#     print(i[1])
    stock_dta.append([ i[0],i[1]])

    
hh=pd.DataFrame(stock_dta,columns=['stocks','k'])

# flatten lists
hh=hh.explode('k')

# flatten dictionaries:
ex_02_df=pd.json_normalize(json.loads(hh.to_json(orient="records")))

ex_02_df

,stocks,k.t,k.o,k.h,k.l,k.c,k.v
0,AAPL,1610570640,131.11,131.120,131.02,131.030,11892
1,AAPL,1610570700,131.05,131.070,130.98,131.050,8640
2,ADBE,1610570640,472.96,472.960,472.80,472.820,819
3,ADBE,1610570700,472.80,472.970,472.80,472.970,910
4,ADI,1610570640,158.68,158.715,158.61,158.610,985
5,ADI,1610570700,158.57,158.595,158.57,158.595,611


# `Rename column names: `

+ What do we notice about the time column by the way?

In [256]:
ex_02_df.rename(columns={"k.t":"time","k.h":"high","k.l":"low",
                         "k.c":"close","k.v":"volume"})

,stocks,time,k.o,high,low,close,volume
0,AAPL,1610570640,131.11,131.120,131.02,131.030,11892
1,AAPL,1610570700,131.05,131.070,130.98,131.050,8640
2,ADBE,1610570640,472.96,472.960,472.80,472.820,819
3,ADBE,1610570700,472.80,472.970,472.80,472.970,910
4,ADI,1610570640,158.68,158.715,158.61,158.610,985
5,ADI,1610570700,158.57,158.595,158.57,158.595,611


# `Wait: we could have done this without a loop`

In [261]:
df_01_better=pd.DataFrame(stocks.items(),columns=['stocks','info']).explode('info')

pd.json_normalize(json.loads(df_01_better.to_json(orient="records")))

,stocks,info.t,info.o,info.h,info.l,info.c,info.v
0,AAPL,1610570640,131.11,131.120,131.02,131.030,11892
1,AAPL,1610570700,131.05,131.070,130.98,131.050,8640
2,ADBE,1610570640,472.96,472.960,472.80,472.820,819
3,ADBE,1610570700,472.80,472.970,472.80,472.970,910
4,ADI,1610570640,158.68,158.715,158.61,158.610,985
5,ADI,1610570700,158.57,158.595,158.57,158.595,611


# `Example 02: What happens if we just do json.normalize?`

In [262]:
df_02=pd.DataFrame(stocks)

In [263]:
# import json

pd.json_normalize(json.loads(df_02.to_json(orient="records")))

,AAPL.t,AAPL.o,AAPL.h,AAPL.l,AAPL.c,AAPL.v,ADBE.t,ADBE.o,ADBE.h,ADBE.l,ADBE.c,ADBE.v,ADI.t,ADI.o,ADI.h,ADI.l,ADI.c,ADI.v
0,1610570640,131.11,131.12,131.02,131.03,11892,1610570640,472.96,472.96,472.8,472.82,819,1610570640,158.68,158.715,158.61,158.610,985
1,1610570700,131.05,131.07,130.98,131.05,8640,1610570700,472.80,472.97,472.8,472.97,910,1610570700,158.57,158.595,158.57,158.595,611


# `What do we notice about Example 02`

+ Can it be improved?
+ Is it a problem?


# `Example 03: More Challenging but,the client wanted this`

+ got me thinking for this one, but I like it.

In [267]:
from collections import defaultdict


mystuff=defaultdict(list)
alt_lst=[]
for key,val in stocks.items():
    for i in val:
        for j in i.items():
            if j[0]=='t' and j[1] not in mystuff['t']:
                mystuff['t'].append(j[1])   
            elif j[0]=='o' and 'c':
                mystuff[key].append(j[1])

my_df=pd.DataFrame(mystuff)

my_df=my_df.rename(columns={"t":"date"})


In [268]:
my_df

,date,AAPL,ADBE,ADI
0,1610570640,131.11,472.96,158.68
1,1610570700,131.05,472.80,158.57


# `Convert date (timestamp) column into human readable`

In [269]:
from datetime import datetime
# datetime.fromtimestamp(my_df['date'][1])

time_Stamp_to_Date=[]

for i in range(len(my_df['date'].values)):
    time_Stamp_to_Date.append(datetime.fromtimestamp(my_df['date'][i]))


In [270]:
my_df['date']=time_Stamp_to_Date

my_df

,date,AAPL,ADBE,ADI
0,2021-01-13 13:44:00,131.11,472.96,158.68
1,2021-01-13 13:45:00,131.05,472.80,158.57


# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

<font size=5>**`Help Support The Channel: Patreon:`**</font><font size=4>[@mrfugudatasci](https://www.patreon.com/mrfugudatasci)</font>

<font size=4>**`Or Consider: Buy Me A Coffee`**[@mrfugudatasci](https://www.patreon.com/mrfugudatasci)</font>